In [ ]:
import pandas as pd


In [ ]:
encdata = pd.read_csv("encounters.csv.gz")

In [ ]:
encdata["AGE@ENC"].plot.hist(bins=50)

In [ ]:
encdata.shape

In [ ]:
encdata.head()

### How many encounters do we have?

In [ ]:
len(encdata['Id_enc'].unique())

In [ ]:
len(encdata['PATIENT'].unique())

### Select BMI Data

In [ ]:
bmi=encdata[encdata['DESCRIPTION_obs']=='Body Mass Index']

In [ ]:
bmi.shape

In [ ]:
bmi = bmi.drop_duplicates(subset='Id_enc', keep='first')
bmi.shape

## Filter based on our alert

In [ ]:
alerts = bmi[(bmi["AGE@ENC"]>45 ) | (bmi["NUMVALUE"]>25)]
alerts

In [ ]:
alerts.shape

## What Percentage of Our Encounters Generate Our Alert?

In [ ]:
alerts.shape[0]/bmi.shape[0]

### How Many Unique Patients Did We Generate an Alert for?

In [ ]:
len(alerts["PATIENT"].unique())